# <center>Прогноз оттока клиентов из телеком-компании</center>
<center> Автор: Роман Сарычев

In [ ]:
from __future__ import (absolute_import, division, print_function, unicode_literals)
import warnings
warnings.simplefilter('ignore')

import pandas as pd
import numpy as np

%pylab inline
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC

from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import train_test_split
from sklearn.learning_curve import validation_curve
from sklearn.learning_curve import learning_curve
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

**[Данные](https://bigml.com/user/francisco/gallery/dataset/5163ad540c0b5e5b22000383) по оттоку клиентов в телком-компании.**

In [ ]:
# Чтение файла
df = pd.read_csv('../../data/telecom_churn.csv')

### <center>Описание набора данных и признаков</center>

In [ ]:
# Проверка, что файл прочитался нормально
df.head().T

Каждая строка представляет собой одного клиента - это **объект** исследования.  
Столбцы - **признаки** объекта.


Описание признаков объекта:  
**State** - Буквенный код штата, номинальный признак  
**Account length** - Общее время, в течение которого клиент обслуживается компанией, количественный признак  
**Area code** - Префикс номера телефона, количественный признак   
**International plan** - Международный роуминг, бинарный признак (подключен/не подключен)  
**Voice mail plan** - Голосовая почта, бинарный признак (подключена/не подключена)  
**Number vmail messages** - Количество голосовых сообщений, количественный признак  
**Total day minutes** - Общая длительность разговоров днем, количественный признак  
**Total day calls** - Общее количество звонков днем, количественный признак  
**Total day charge** - Общая сумма оплаты за услуги днем, количественный признак  
**Total eve minutes** - Общая длительность разговоров вечером, количественный признак  
**Total eve calls** - Общее количество звонков вечером, количественный признак  
**Total eve charge** - Общая сумма оплаты за услуги вечером, количественный признак  
**Total night minutes** - Общая длительность разговоров ночью, количественный признак  
**Total night calls** - Общее количество звонков ночью, количественный признак  
**Total night charge** - Общая сумма оплаты за услуги ночью, количественный признак  
**Total intl minutes** - Общая длительность международных разговоров, количественный признак  
**Total intl calls** - Общее количество  международных разговоров, количественный признак  
**Total intl charge** -  Общая сумма оплаты за международные разговоры, количественный   признак  
**Customer service calls** - Количество обращений в сервисный центр, количественный признак 
  
Целевая переменная: **Churn** - Признак оттока, бинарный признак (1 - потеря клиента, то есть отток)  

### <center>Описание предобработки данных</center>

In [ ]:
# Тип данных признаков 'International plan' и 'Voice mail plan' - объекты,
# нужно преобразовать  в булевый тип.
obj_cols = ['International plan', 'Voice mail plan']
df[obj_cols] = df[obj_cols] == 'Yes'

In [ ]:
# Преобразование номинального признака названия штата в количественный
state_encoder = preprocessing.LabelEncoder()
state_encoder.fit(df['State'])
df['State'] = state_encoder.transform(df['State']).astype("float64")

###  <center>Первичный анализ признаков</center>

In [ ]:
#Просмотр типов данных
df.info()

# Все данные заполнены, пропусков нет.

In [ ]:
# Смотрим на статистические характеристики:
df.describe().T

In [ ]:
# Распределение целевой переменной
df['Churn'].hist()

In [ ]:
df['Churn'].value_counts()

Выборка не сбалансированна, одного класса больше чем другого.

### <center>Первичный визуальный анализ признаков </center>

In [ ]:
# Анализируемые признаки (переменная создана для удобства предварительного анализа)
predictors = [
#  'State',
 'Account length',
#  'Area code',
# 'International plan',
# 'Voice mail plan',
 'Number vmail messages',
 'Total day minutes',
 'Total day calls',
 'Total day charge',
 'Total eve minutes',
 'Total eve calls',
 'Total eve charge',
 'Total night minutes',
 'Total night calls',
 'Total night charge',
 'Total intl minutes',
 'Total intl calls',
 'Total intl charge',
 'Customer service calls',
#  'Churn'
]

In [ ]:
# Ищем коррелирующие признаки
corr = df[predictors].corr()
sns.heatmap(corr)

In [ ]:
# Строим графики распределения признаков
plots = df[predictors].hist(figsize=(12,10))

### <center>Закономерности:</center>
1. На первый взгляд величина среднего количества звонков в разное время суток наблюдается на одном уровне, средняя продолжительность звонков вечером и ночью в среднем все-таки больше. Это соответствует действительности и логично, поскольку люди чаще совершают продолжительные звонки в свободное от работы и учебы время, а днем большинство совершаются кратковременные деловые звонки.
2. Несмотря на то что средняя продолжительность разговоров увеличивается по мере смены времени суток (вечером и ночью больше, чем днем), мы можем наблюдать, что среднее количество звонков остается на прежнем уровне. Но при этом сумма оплаты разговоров снижается, что, вероятно, связанно с повременным тарифом, т.е. в разное время суток, разная тарификация.
3. Средняя продолжительность международных звонков небольшая. Это связанно с дорогими тарифами на роуминг.

In [ ]:
# Пробуем найти влияние количества звонков, общей продолжительности и суммы оплаты.
# Берем вечерний период, так как он самый активный. 
sns.pairplot(df[['Total eve minutes', 'Total eve calls', 'Total eve charge',
                 'Churn']], hue='Churn')

# Зависимости не выявлено, отток клиентов по этим параметрам равномерен.
# Наблюдается линейная зависимость суммы оплаты от общей продолжительности разговоров.

Основываясь на экспертном мнении, можно предположить, что на отток клиента может сильно влиять общее время обслуживания клиента и количество обращений в сервисный центр. Пробуем выявить зависимость.

In [ ]:
# Зависимость признака оттока от количества обращений в сервисный центр
sns.countplot(x='Customer service calls', hue="Churn", data=df)

Подсчитаем долю вышедших в отток клиентов от не вышедших и
долю вышедших в отток от суммы всех клиентов в определенном количестве звонков.

Построим линейные графики.

In [ ]:
true_service_calls = df[df['Churn'] == True].groupby('Customer service calls')['Churn'].count()
false_service_calls = df[df['Churn'] == False].groupby('Customer service calls')['Churn'].count()
ratio = true_service_calls / false_service_calls * 100
ratio_all = true_service_calls / (true_service_calls + false_service_calls) * 100

In [ ]:
plt.plot(ratio)
plt.plot(ratio_all)
plt.xlabel('Customer service calls')
plt.ylabel('Churn')
plt.show()

In [ ]:
# Так как продолжительность является количественным и непрерывным признаком,
# то для упрощения визуализации разделим клиентов на несколько 'поколений'.
df['Generation'] = df['Account length'].apply(lambda x: x//30)
sns.countplot(x='Generation', hue="Churn", data=df)

Подсчитаем доли вышедших в отток клиентов от не вышедших и
долю вышедших в отток от суммы всех клиентов по поколениям.

Построим линейные графики.

In [ ]:
true_generation = df[df['Churn'] == True].groupby('Generation')['Churn'].count()
false_generation = df[df['Churn'] == False].groupby('Generation')['Churn'].count()
ratio = true_generation / false_generation * 100
ratio_all = true_generation / (true_generation + false_generation) * 100

In [ ]:
plt.plot(ratio)
plt.plot(ratio_all)
plt.xlabel('Generation')
plt.ylabel('Churn')
plt.show()

### <center>Инсайты:</center>
1. Клиенты, совершающие более 3-х звонков в call-центр, имеют разительно  более высокий процент оттока, Это может быть обусловлено тем, что большое кол-во звонков объясняется существованием серьезных проблем у звонящих клиентов, что сильно повышает вероятность их попадания в отток.  
В случаях с меньшим количеством звонков, чем 3, процент оттока остается стабильным на достаточно низком уровне (10%-15%), что подтверждает приведенную выше гипотезу.
2. Анализ единиц продолжительности жизни клиента показал, что увеличение данного показателя ведет к более интенсивному оттоку клиентов. 
Данная закономерность ожидаема и может говорить о том, что рассматриваемый клиентский продукт подвержен изменениям "моды".  
C шестого этапа жизни (из выделенных периодов в 30 единиц), заметно значительное увеличение интенсивности оттока.  
Это может быть обусловлено тем, что на этот момент клиент приобретает определенное отрицательное благо, либо теряет положительное.  
Например, теряет льготные условия обслуживания, которые предоставлялись ему в течение 6 этапов и т. п. 



### <center>Создание новых признаков</center>

Звонки по телефону - это целевая услуга, которая предоставляется абоненту,
поэтому пробуем рассчитать, сколько стоит гипотетическая "минута" (то есть без учета других услуг) разговора у абонента.

In [ ]:
df['Cost of Minute'] = (df['Total day charge'] + 
                        df['Total eve charge'] +
                        df['Total night charge'] +
                        df['Total intl charge']) / (df['Total day minutes'] + 
                                                    df['Total eve minutes'] +
                                                    df['Total night minutes'] +
                                                    df['Total intl minutes'])

In [ ]:
# Визуальный анализ нового признака
df['round'] = df['Cost of Minute'].apply(lambda x: round(x, 2) * 100)
sns.countplot(x='round', hue="Churn", data=df)
plt.xlabel('Cents in a minute')

In [ ]:
true_generation = df[df['Churn'] == True].groupby('round')['Churn'].count()
false_generation = df[df['Churn'] == False].groupby('round')['Churn'].count()
ratio = true_generation / false_generation * 100
ratio_all = true_generation / (true_generation + false_generation) * 100

In [ ]:
plt.plot(ratio)
plt.plot(ratio_all)
plt.xlabel('Cents in a minute')
plt.ylabel('Churn')
plt.show()

Анализ стоимости "минуты" показывает, что признак имеет важное значение.
При стоимости минуты больше 10 центов, вероятность попадания абонента в отток возрастает,
а при стоимости менее 7 центов за минуту - вероятность очень мала.

### <center>Отбор признаков</center>

In [ ]:
# Маштабируем переменные и конвертируем назад в Pandas DataFrame
df_scale = preprocessing.scale(df)
df_scale = pd.DataFrame(df_scale)
df_scale.columns = df.columns

Для оценки важности признаков сделаем предсказание случайного леса с параметрами по умолчанию. Вместо кросс-валидации будем использовать Out-of-Bag оценку.

In [ ]:
# Выделим обучающую выборку и целевую переменую 
X, y = df[[s for s in df_scale.columns if s != 'Churn']], df['Churn']

In [ ]:
first_forest = RandomForestClassifier(n_estimators=1000, max_depth = 5, 
                                      oob_score=True, n_jobs=-1,
                                      random_state=42).fit(X, y)

In [ ]:
# Посмотрим точность предсказания
first_forest.oob_score_

In [ ]:
first_forest_predictions = first_forest.predict(X)
features = pd.DataFrame(first_forest.feature_importances_, index=X.columns,
                        columns=['Importance']).sort(['Importance'], ascending=False)
features

In [ ]:
# Кривая оценок важности признаков
plt.plot(range(len(features.Importance.tolist())), 
         features.Importance.tolist())

Анализируя оценки важности можно сделать следующие выводы:
1. Ожидаемо вспомогаемые признаки Generation и round имеют малый вес. Также ожидаемо, что код штата и префикс номера имеют малый вес. Эти признаки не будут использоваться в обучающей выборке.
2. Показатели активности абонента с маленькими значениями, например: Account length, Total day calls и другие, все же могут вносить незначительные коррективы, поэтому удалятся не будут.

In [ ]:
X = df[['Total day charge',
        'Total day minutes',
        'Customer service calls',
        'International plan',
        'Total eve minutes',
        'Total eve charge',
        'Cost of Minute',
        'Number vmail messages',
        'Total intl calls',
        'Total intl minutes',
        'Voice mail plan',
        'Total intl charge',
        'Total night minutes',
        'Total night charge',
        'Total day calls',
        'Total night calls',
        'Account length',
        'Total eve calls']]

### <center>Построение модели классификации</center>

In [ ]:
# Разбиваем на тестовую и обучающую выборку
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

Попробуем четыре разных классификатора: логистическую регрессию, метод ближайших соседей, Gradient boosting, Random Forest и SVM. Так как у нас сильный дисбаланс в выборке, то в качестве меры будем использовать F1 score.

In [ ]:
classifiers = [LogisticRegression(),
               KNeighborsClassifier(),
               GradientBoostingClassifier(), 
               RandomForestClassifier(), 
               SVC()]
classifiers_name = ['LogisticRegression',
                    'KNeighborsClassifier',
                    'GradientBoostingClassifier', 
                    'RandomForestClassifier', 
                    'SVC']

In [ ]:
# Настройка параметров выбранных алгоритмов с помощью GridSearchCV 
n_folds = 5
scores = []
fits = []
logistic_params = {'penalty': ('l1', 'l2'),
                   'C': (.01,.1,1,5)}
knn_params = {'n_neighbors': list(range(3, 12, 2))}
gbm_params = {'n_estimators': [100, 300, 500],
              'learning_rate':(0.1, 0.5, 1),
              'max_depth': list(range(3, 6)), 
              'min_samples_leaf': list(range(10, 31, 10))}
forest_params = {'n_estimators': [100, 300, 500],
                 'criterion': ('gini', 'entropy'), 
                 'max_depth': list(range(3, 6)), 
                 'min_samples_leaf': list(range(10, 31, 10))}

svm_param = {'kernel' : ('linear', 'rbf'), 'C': (.5, 1, 2)}
params = [logistic_params, knn_params, gbm_params, forest_params, svm_param]

### <center>Кросс-валидация</center>

In [ ]:
for i, each_classifier in enumerate(classifiers):
    clf = each_classifier
    clf_params = params[i]
    grid = GridSearchCV(clf, clf_params, 
                        cv=StratifiedKFold(y_train, n_folds=n_folds,
                        shuffle=False, random_state=42), 
                        n_jobs=-1, scoring="f1")
    grid.fit(X_train, y_train)
    fits.append(grid.best_params_)
    clf_best_score = grid.best_score_
    scores.append(clf_best_score)
    print(classifiers_name[i], clf_best_score, "\n", grid.best_params_, "\n")

In [ ]:
grid_value = max(scores)
grid_index = [i for i in range(len(scores)) if scores[i]==grid_value][0]
print("Лучший классификатор при GridSearch:",
      classifiers_name[grid_index], grid_value)
print(fits[grid_index])

In [ ]:
clf_params = {'n_estimators': (300, 350, 400), 
              'learning_rate': (0.1, 0.3, 0.5, 0.75, 1), 
              'min_samples_leaf': list(range(1, 14, 3))}

clf = classifiers[grid_index]
grid = GridSearchCV(clf, clf_params, cv=n_folds, 
                    n_jobs=-1, scoring="f1")
grid.fit(X_train, y_train)
clf_best_score = grid.best_score_
clf_best_params = grid.best_params_
clf_best = grid.best_estimator_
mean_validation_scores = []
print("Лучший результат", clf_best_score, 
      "лучшие параметры", clf_best_params)

### <center>Построение кривых валидации и обучения</center>

In [ ]:
def plot_with_std(x, data, **kwargs):
        mu, std = data.mean(1), data.std(1)
        lines = plt.plot(x, mu, '-', **kwargs)
        plt.fill_between(x, mu - std, mu + std, edgecolor='none',
                         facecolor=lines[0].get_color(), alpha=0.2)
        
def plot_learning_curve(clf, X, y, scoring, cv=5):
 
    train_sizes = np.linspace(0.05, 1, 20)
    n_train, val_train, val_test = learning_curve(clf,
                                                  X, y, train_sizes, cv=cv,
                                                  scoring=scoring)
    plot_with_std(n_train, val_train, label='training scores', c='green')
    plot_with_std(n_train, val_test, label='validation scores', c='red')
    plt.xlabel('Training Set Size'); plt.ylabel(scoring)
    plt.legend()

def plot_validation_curve(clf, X, y, cv_param_name, 
                          cv_param_values, scoring):

    val_train, val_test = validation_curve(clf, X, y, cv_param_name,
                                           cv_param_values, cv=5,
                                                  scoring=scoring)
    plot_with_std(cv_param_values, val_train, 
                  label='training scores', c='green')
    plot_with_std(cv_param_values, val_test, 
                  label='validation scores', c='red')
    plt.xlabel(cv_param_name); plt.ylabel(scoring)
    plt.legend()

In [ ]:
# Кривая обучения
plot_learning_curve(GradientBoostingClassifier(n_estimators=2, 
                    learning_rate=1.5, min_samples_leaf=7),
                   X_train, y_train, scoring='f1', cv=10)

In [ ]:
# Кривая валидации
learning_rates = np.linspace(0.1, 2.3, 20)
plot_validation_curve(GradientBoostingClassifier(n_estimators=250, 
                    min_samples_leaf=7), X_train, y_train, 
                    cv_param_name='learning_rate', 
                    cv_param_values=learning_rates,
                    scoring='f1')

### <center>Финальный прогноз для отложенной выборки</center>

In [ ]:
final_gbm = GradientBoostingClassifier(n_estimators=300, 
                    min_samples_leaf=10, learning_rate=0.1, max_depth=4)
final_gbm.fit(X_train, y_train)
final_pred = final_gbm.predict(X_test)
accuracy_score(y_test, final_pred), f1_score(y_test, final_pred)

### <center>Оценка модели с описанием выбранной метрики</center>

Построена модель предсказания, уйдет ли абонент телеком-копмании в отток. Модель предсказывает с 96%-ной долей правильных ответов на отложенных 30% выборки. Но accuracy не очень хорошо характеризует качество модели из-за сильного дисбаланса в целевой переменной (~85% против ~15%), поэтому в качестве целевой была выбрана метрика F1-score. На отложенной выборке удалось добиться хорошего результата F1=0.82. Построены кривые обучения и валидационные кривые. Видно, что увеличение количества примеров более 1700 не приносит существенной выгоды (у нас в обучающей выборке более 3000 примеров).

## <center>Общие выводы</center>

По результатам проведенного анализа, можно увидеть, что есть определенные зависимости и признаки показателя оттока. 
Наблюдаются определенные закономерности - продолжительность звонков в определенное время суток, международные звонки, сумма оплаты разговоров. 
Имеет влияние количество обращений клиентами в сервисный центр - совершающие более 3-х звонков с большей вероятностью попадут в отток. 
Также прямое отношение в показателю оттока имеет срок жизни клиента - клиенты со сроком жизни более 6-ти отрезков времени по 30 единиц имеют больший риск попасть в отток. 
Выявленным признаком оттока является стоимость минуты разговора клиента - минимальная вероятность попадания клиента в отток наблюдается при стоимости минуты разговора клиента при 7 центах. С наибольшей вероятностью клиент покинет компанию при стоимости в 10 центов и более.